# SQL Murder Mystery - Overview

![Sql Murder Mystery image](http://mystery.knightlab.com/174092-clue-illustration.png)

There's been a Murder in SQL City! The SQL Murder Mystery is designed to be both a self-directed lesson to learn SQL concepts and commands and a fun game for experienced SQL users to solve an intriguing crime.

A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a murder that occurred sometime on **Jan.15, 2018** and that it took place in **SQL City**. Start by retrieving the corresponding crime scene report from the police department’s database.

You can solve the mystery on their website (http://mystery.knightlab.com/) itself or download it from the github repository (https://github.com/NUKnightLab/sql-mysteries) and solve it in your preferred method.

I have uploaded the database file in the kaggle dataset section - https://www.kaggle.com/datasets/johnp47/sql-murder-mystery-database

**Below is the schema of the database -**

![Schema](https://raw.githubusercontent.com/NUKnightLab/sql-mysteries/master/schema.png)

# Import File

In [ ]:
#I will be using python's sqlite3 library to solve the mystery
#importing libraries
import pandas as pd
import sqlite3 as sql

# Connection

In [ ]:
!rm -Rf /content/__MACOSX
!wget https://liviaellen.com/wp-content/uploads/2024/04/sql_db.zip
!unzip sql_db.zip
!mv /content/sql_db/sql-murder-mystery.db sql-murder-mystery.db
!rm -f sql_db.zip
!rm -Rf /content/__MACOSX
!rm -Rf /content/sql_db/

--2024-06-08 14:30:44--  https://liviaellen.com/wp-content/uploads/2024/04/sql_db.zip
Resolving liviaellen.com (liviaellen.com)... 54.158.147.214
Connecting to liviaellen.com (liviaellen.com)|54.158.147.214|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1853895 (1.8M) [application/zip]
Saving to: ‘sql_db.zip’

sql_db.zip          100%[===================>]   1.77M  --.-KB/s    in 0.1s    

2024-06-08 14:30:44 (13.4 MB/s) - ‘sql_db.zip’ saved [1853895/1853895]

Archive:  sql_db.zip
   creating: sql_db/
  inflating: sql_db/sql-murder-mystery.db  
  inflating: __MACOSX/sql_db/._sql-murder-mystery.db  


In [ ]:
#setting up a connection to the database
con = sql.connect('sql-murder-mystery.db')

# Let's Solve it!

In [ ]:


 #looking for table names
query_0 = '''
SELECT name
  FROM sqlite_master
 where type = 'table'
'''
#seting the dataframe width to max
pd.set_option('display.max_colwidth', None)
#running our query
pd.read_sql_query(query_0, con)

,name
0,crime_scene_report
1,drivers_license
2,facebook_event_checkin
3,interview
4,get_fit_now_member
5,get_fit_now_check_in
6,solution
7,income
8,person


In [ ]:
#running our first query, looking for data from crime scene report
query_1 = '''
SELECT *
FROM crime_scene_report
WHERE type = "murder" AND city ="SQL City" AND date="20180115"
'''
#seting the dataframe width to max
pd.set_option('display.max_colwidth', None)
#running our query
pd.read_sql_query(query_1, con)

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


*Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave"*

Got our crime scene report, as per the report there are 2 witnesses. Let's, look for them.

In [ ]:
#checking personal details of both the witnesses
query_2 = '''
SELECT * FROM PERSON
WHERE address_street_name="Northwestern Dr"
ORDER BY address_number DESC LIMIT 1
'''

pd.read_sql_query(query_2, con)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [ ]:
query_3 = '''
SELECT * FROM PERSON
WHERE address_street_name="Franklin Ave"
AND name LIKE "%Annabel%"

'''

#16371
pd.read_sql_query(query_3, con)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [ ]:
#lets view the interview of both the witnesses taken after the murder.
query_4 = '''
SELECT * FROM interview
WHERE person_id in (16371,14887	)
'''
pd.read_sql_query(query_4, con)

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


*So, we got 2 clues-*
* *Killer is a man and a member of the gym with a status of gold and having a membership no. starting with 48Z and left in a car with a no. plate of H42W*
* *He was working out in the gym on 9th of Jan*

In [ ]:
#Checking the gym database with above details
query_5 = '''
SELECT *
  FROM get_fit_now_member
  WHERE id LIKE "48Z%" AND membership_status="gold"

'''
pd.read_sql_query(query_5, con)



,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


*Two member's found and their membership id*

In [ ]:
#now, let's check the car details by the above details
query_6 = '''
SELECT *
  FROM drivers_license WHERE plate_number LIKE "%H42W%"

'''
pd.read_sql_query(query_6, con)



,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
1,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
2,664760,21,71,black,black,male,4H42WR,Nissan,Altima


*Two male with a plate no. containg H42W*

In [ ]:
#checking personal details of both the males from the above query
query_7 = '''
SELECT* FROM PERSON
WHERE license_id in (183779,	423327,	664760	)

'''
pd.read_sql_query(query_7, con)

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279
2,78193,Maxine Whitely,183779,110,Fisk Rd,137882671


In [ ]:
#lets check which of this two are a member of the gym?
query_8 = '''
SELECT * FROM get_fit_now_check_in
WHERE check_in_date = "20180109"

'''
pd.read_sql_query(query_8, con)

,membership_id,check_in_date,check_in_time,check_out_time
0,X0643,20180109,957,1164
1,UK1F2,20180109,344,518
2,XTE42,20180109,486,1124
3,1AE2H,20180109,461,944
4,6LSTG,20180109,399,515
5,7MWHJ,20180109,273,885
6,GE5Q8,20180109,367,959
7,48Z7A,20180109,1600,1730
8,48Z55,20180109,1530,1700
9,90081,20180109,1600,1700


**Finally, found the murderer - Jeremy Bowers.**
*Both the membership id and status also matches as per the information we found earlier.*

Now to confirm you can visit the website and run this query

> INSERT INTO solution VALUES (1, 'Jeremy Bowers');
        SELECT value FROM solution;
        
**Result shown is:**

*Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer.*

In [ ]:
#There's more to this, reading the transcript of the murderer
query_9 = '''
SELECT * FROM interview
WHERE person_id in (67318)

'''
pd.read_sql_query(query_9, con)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


*So, the real villain is a woman with a Tesla car and red hair.*
*Using the above clues let find out who's the mastermind behind this murder.*

In [ ]:
query_10 = '''
SELECT person_id, count(*) AS number_row FROM facebook_event_checkin
WHERE event_name='SQL Symphony Concert'
AND date LIKE '201712%'
GROUP BY person_id
HAVING number_row=3
'''
pd.read_sql_query(query_10, con)

,person_id,number_row
0,24556,3
1,99716,3


*Three woman with Tesla Model S and red hair color*

In [ ]:
#personal details of the above three woman are:
query_11 = '''
SELECT * from PERSON WHERE id in (99716	,24556	)
'''
pd.read_sql_query(query_11, con)

,id,name,license_id,address_number,address_street_name,ssn
0,24556,Bryan Pardo,101191,703,Machine Ln,816663882
1,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [ ]:
#checking the event SQL symphony concert
query_12 = '''
SELECT * from drivers_license WHERE id in (202298	,101191	)

'''
pd.read_sql_query(query_12, con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S


**Finally, found the mastermind/real villian of this whole mystry - Miranda Priestly**

Now, confirming it on the website shows us-

> INSERT INTO solution VALUES (1, 'Miranda Priestly');
        SELECT value FROM solution;
        
**Result is:**
*Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!*